In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import pymc3 as pm
import arviz as az

sns.set()


In [ ]:
abbs = ['SWE','FIN','NOR','DNK','ISL']

In [ ]:
df = pd.DataFrame()

for a in abbs:
    temp = pd.read_excel('mortality_org.xlsx',sheet_name=a,skiprows=2)
    
    df = pd.concat([df,temp],axis=0)

In [ ]:
df_2020 = (df[(df.loc[:,'Sex'] == 'b') & (df['Year'] == 2020)]).copy()

df_2020.rename(columns={'0-14.1': 'age_grp_0-14','15-64.1' : 'age_grp_15-64',
                      '65-74.1' : 'age_grp_65-74', '75-84.1' : 'age_grp_75-84',
                      '85+.1' : 'age_grp_85+','Total.1' : 'all_age_grp'},inplace=True)
df_2020

In [ ]:
df_timeline = (df[(df.loc[:,'Sex'] == 'b')]).copy()
df_timeline.rename(columns={'0-14.1': 'age_grp_0-14','15-64.1' : 'age_grp_15-64',
                      '65-74.1' : 'age_grp_65-74', '75-84.1' : 'age_grp_75-84',
                      '85+.1' : 'age_grp_85+','Total.1' : 'all_age_grp'},inplace=True)
df_timeline

In [ ]:
def year_week_2_date(y,w):
    return pd.to_datetime(str(y) + ' ' + str(w) + ' 0',format='%Y %W %w')


df_timeline.index = df_timeline.apply(lambda row : year_week_2_date(row.Year,row.Week),axis=1)
df_timeline

In [ ]:
country_grp = df_timeline.groupby('Country')
swe = country_grp.get_group('SWE').resample('Y').mean().loc['2000-01-01' :]
fin = country_grp.get_group('FIN').resample('Y').mean().loc['2000-01-01' :]
den = country_grp.get_group('DNK').resample('Y').mean().loc['2000-01-01' :]
nor = country_grp.get_group('NOR').resample('Y').mean().loc['2000-01-01' :]
isl = country_grp.get_group('ISL').resample('Y').mean().loc['2000-01-01' :]

swe

In [ ]:
title = 'Nordic countries : trend overall mortality rate -2020 YTD\nDataSource : mortality.org'

ax = swe['all_age_grp'].plot(style='yo--',figsize=(18,12),label='SWE',title=title)
fin['all_age_grp'].plot(style='bo--',ax=ax,label='FIN')
den['all_age_grp'].plot(style='ro--',ax=ax,label='DNK')
nor['all_age_grp'].plot(style='ko--',ax=ax,label='NOR')
isl['all_age_grp'].plot(style='go--',ax=ax,label='ISL')

ax.set_ylabel('mortality')

ax.set_ylim([0,0.012])
plt.legend(loc='upper left')
plt.savefig('nordic_mortality_trend.jpg',format='jpg')

In [ ]:
swe_2020 = df_2020.loc[df_2020['Country'] == 'SWE']
fin_2020 = df_2020.loc[df_2020['Country'] == 'FIN']
den_2020 = df_2020.loc[df_2020['Country'] == 'DNK']
nor_2020 = df_2020.loc[df_2020['Country'] == 'NOR']
isl_2020 = df_2020.loc[df_2020['Country'] == 'ISL']

ax = swe_2020.plot(x='Week',y='all_age_grp',style=['o--'],label='SWE',figsize=(18,12),
             title='2020 Weekly Overall Death Rate Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='all_age_grp',style=['o--'],label='FIN',ax=ax)
den_2020.plot(x='Week',y='all_age_grp',style=['o--'],label='DNK',ax=ax)
nor_2020.plot(x='Week',y='all_age_grp',style=['o--'],label='NOR',ax=ax)
isl_2020.plot(x='Week',y='all_age_grp',style=['yo--'],label='ISL',ax=ax)



plt.ylabel('Weekly Death Rate')
plt.savefig('mortality_org_overall_death_rate_nor_2020dics.jpg',format='jpg')

In [ ]:
swe_2020.tail()

In [ ]:
ax = swe_2020.plot(x='Week',y=['age_grp_0-14','age_grp_15-64','age_grp_65-74',
                         'age_grp_75-84','age_grp_85+'],style=['o--'] * 5,figsize=(18,12),
             title='SWE 2020 Weekly Death Rate Nordic Countries\nDataSource : mortality.org')

plt.ylabel('Weekly Death Rate')

In [ ]:
ax = fin_2020.plot(x='Week',y=['age_grp_0-14','age_grp_15-64','age_grp_65-74',
                         'age_grp_75-84','age_grp_85+'],style=['o--'] * 5,figsize=(18,12),
             title='FIN 2020 Weekly Mortality Nordic Countries\nDataSource : mortality.org')

In [ ]:
ax = den_2020.plot(x='Week',y=['age_grp_0-14','age_grp_15-64','age_grp_65-74',
                         'age_grp_75-84','age_grp_85+'],style=['o--'] * 5,figsize=(18,12),
             title='DNK 2020 Weekly Mortality Nordic Countries\nDataSource : mortality.org')

In [ ]:
ax = nor_2020.plot(x='Week',y=['age_grp_0-14','age_grp_15-64','age_grp_65-74',
                         'age_grp_75-84','age_grp_85+'],style=['o--'] * 5,figsize=(18,12),
             title='NOR 2020 Weekly Mortality Nordic Countries\nDataSource : mortality.org')

In [ ]:
ax = isl_2020.plot(x='Week',y=['age_grp_0-14','age_grp_15-64','age_grp_65-74',
                         'age_grp_75-84','age_grp_85+'],style=['o--'] * 5,figsize=(18,12),
             title='ISL 2020 Weekly Mortality Nordic Countries\nDataSource : mortality.org')

In [ ]:
ax = swe_2020.plot(x='Week',y='age_grp_0-14',style='o--',figsize=(18,12),label='SWE 0-14',
             title='2020 Weekly Death Rate per age group Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='age_grp_0-14',style='o--',ax=ax,label='FIN 0-14')
den_2020.plot(x='Week',y='age_grp_0-14',style='o--',ax=ax,label='DNK 0-14')
nor_2020.plot(x='Week',y='age_grp_0-14',style='o--',ax=ax,label='NOR 0-14')
isl_2020.plot(x='Week',y='age_grp_0-14',style='yo--',ax=ax,label='ISL 0-14')

plt.ylabel('Weekly Death Rate')
plt.savefig ('mortality_org_0_14.jpg',format='jpg')

In [ ]:
ax = swe_2020.plot(x='Week',y='age_grp_15-64',style='o--',figsize=(18,12),label='SWE 15-64',
             title='2020 Weekly Death Rate per age group Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='age_grp_15-64',style='o--',ax=ax,label='FIN 15-64')
den_2020.plot(x='Week',y='age_grp_15-64',style='o--',ax=ax,label='DNK 15-64')
nor_2020.plot(x='Week',y='age_grp_15-64',style='o--',ax=ax,label='NOR 15-64')
isl_2020.plot(x='Week',y='age_grp_15-64',style='yo--',ax=ax,label='ISL 15-64')

plt.ylabel('Weekly Death Rate')

plt.savefig('mortality_org_15_64.jpg',format='jpg')

In [ ]:
ax = swe_2020.plot(x='Week',y='age_grp_65-74',style='o--',figsize=(18,12),label='SWE 65-74',
             title='2020 Weekly Death Rate per age group Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='age_grp_65-74',style='o--',ax=ax,label='FIN 65-74')
den_2020.plot(x='Week',y='age_grp_65-74',style='o--',ax=ax,label='DNK 65-74')
nor_2020.plot(x='Week',y='age_grp_65-74',style='o--',ax=ax,label='NOR 65-74')
isl_2020.plot(x='Week',y='age_grp_65-74',style='yo--',ax=ax,label='ISL 65-74')

plt.ylabel('Weekly Death Rate')

plt.savefig('mortality_org_65_74.jpg',format='jpg')

In [ ]:
ax = swe_2020.plot(x='Week',y='age_grp_75-84',style='o--',figsize=(18,12),label='SWE 75-84',
             title='2020 Weekly Death Rate per age group Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='age_grp_75-84',style='o--',ax=ax,label='FIN 75-84')
den_2020.plot(x='Week',y='age_grp_75-84',style='o--',ax=ax,label='DNK 75-84')
nor_2020.plot(x='Week',y='age_grp_75-84',style='o--',ax=ax,label='NOR 75-84')
isl_2020.plot(x='Week',y='age_grp_75-84',style='yo--',ax=ax,label='ISL 75-84')

plt.ylabel('Weekly Death Rate')

plt.savefig('mortality_org_75_84.jpg',format='jpg')

In [ ]:
ax = swe_2020.plot(x='Week',y='age_grp_85+',style='o--',figsize=(18,12),label='SWE 85+',
             title='2020 Weekly Death Rate per age group Nordic Countries\nDataSource : mortality.org')

fin_2020.plot(x='Week',y='age_grp_85+',style='o--',ax=ax,label='FIN 85+')
den_2020.plot(x='Week',y='age_grp_85+',style='o--',ax=ax,label='DNK 85+')
nor_2020.plot(x='Week',y='age_grp_85+',style='o--',ax=ax,label='NOR 85+')
isl_2020.plot(x='Week',y='age_grp_85+',style='yo--',ax=ax,label='ISL 85+')

plt.ylabel('Weekly Death Rate')

plt.savefig('mortality_org_85+.jpg',format='jpg')

In [ ]:
swe_2020['Total'].sum() / (10.23e6 * 37/52)



In [ ]:
swe_2020

In [ ]:
swe_2020['Total'] / 10.23e6 *52

In [ ]:
def age_grp_mean(col):
    return col.mean()

age_grp_mean(swe_2020['age_grp_0-14'])
age_grp_mean(swe_2020['age_grp_15-64'])
age_grp_mean(swe_2020['age_grp_65-74'])
age_grp_mean(swe_2020['age_grp_75-84'])
age_grp_mean(swe_2020['age_grp_85+'])

In [ ]:
df_2020

In [ ]:
fig,axes = plt.subplots(5,1,figsize=(18,12),sharex=True)


title='Nordic Countries 2020 death rate distribution, age group 0-14'
sns.violinplot(x='Country',y='age_grp_0-14',data=df_2020,ax=axes[0])
axes[0].set_title(title)
axes[0].set_ylabel('death rate')

title='Nordic Countries 2020 death rate distribution, age group 15-64'
sns.violinplot(x='Country',y='age_grp_15-64',data=df_2020,ax=axes[1])
axes[1].set_title(title)
axes[1].set_ylabel('death rate')

title='Nordic Countries 2020 death rate distribution, age group 65-74'
sns.violinplot(x='Country',y='age_grp_65-74',data=df_2020,ax=axes[2])
axes[2].set_title(title)
axes[2].set_ylabel('death rate')

title='Nordic Countries 2020 death rate distribution, age group 75-84'
sns.violinplot(x='Country',y='age_grp_75-84',data=df_2020,ax=axes[3])
axes[3].set_title(title)
axes[3].set_ylabel('death rate')

title='Nordic Countries 2020 death rate distribution, age group 85+'
sns.violinplot(x='Country',y='age_grp_85+',data=df_2020,ax=axes[4])
axes[4].set_title(title)
axes[4].set_ylabel('death rate')

plt.tight_layout()

plt.savefig('nor_2020dic_death_rate_distr.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Nordic countries 2020 weekly death rate distribution, all age groups')
sns.violinplot(x='Country',y='all_age_grp',data=df_2020)
plt.ylabel('death rate distribution')

plt.savefig('nor_2020dic_death_distr_all_ages.jpg',format='jpg')

In [ ]:

transaction_df = df_2020[['Country','age_grp_0-14','age_grp_15-64']].copy()
#transaction_df['age_grp_0-14'].apply()

In [ ]:
tips = sns.load_dataset('tips')
tips

In [ ]:
df_timeline = df_timeline.loc['2007-01-01' :]
df_timeline

In [ ]:
unique_countries = df_timeline['Country'].unique()
country_idx_map = dict(zip(unique_countries,range(len(unique_countries))))
country_idx_map

In [ ]:
unique_dates = np.unique(df_timeline.index)
date_idx_map = dict(zip(pd.to_datetime(unique_dates),range(len(unique_dates))))


In [ ]:
df_timeline['country_idx'] = df_timeline['Country'].apply(lambda x : country_idx_map[x])
df_timeline

In [ ]:
week_idx = df_timeline.reset_index()['index'].apply(lambda x : date_idx_map[x])

In [ ]:
df_timeline['week_idx'] = week_idx.values
df_timeline

In [ ]:
df_timeline.loc[df_timeline['Country'] == 'NOR']

In [ ]:
plt.scatter(df_timeline.loc[df_timeline['Country'] == 'SWE']['week_idx'],
            df_timeline.loc[df_timeline['Country']=='SWE']['all_age_grp'])

In [ ]:

x = df_timeline['week_idx'].values
x

In [ ]:
def standardize(series):
    return (series - series.mean()) / series.std()

def regression(outcome='all_age_grp'):

    week_idx = df_timeline['week_idx'].values
    country_idx = df_timeline['country_idx'].values

    scale_outcome = 1000

    model = pm.Model()
    with model:

        alpha = pm.Normal('alpha',0,1,shape=len(unique_countries))
        beta = pm.Normal('beta',0, 1,shape=len(unique_countries))

        sigma = pm.Uniform('sigma',0,5)

        mu = pm.Deterministic('mu',alpha[country_idx] + standardize(x) * beta[country_idx])

        obs = pm.Normal('obs',mu,sigma,observed=df_timeline[outcome] * scale_outcome) 

        trace = pm.sample(2000,tune=2000)
        
        return trace

In [ ]:
df_timeline.head()

In [ ]:
outcome = 'age_grp_85+'

trace=regression(outcome)

In [ ]:
with model:
    print (az.summary(trace,var_names=['~mu'],hdi_prob=0.89))

In [ ]:
with model:
    az.plot_posterior(trace,var_names=['beta'],hdi_prob=0.89)

In [ ]:
def plot_reg(outcome):
    
    fin_beta = trace['beta'][:,1].mean() / scale_outcome
    fin_alpha = trace['alpha'][:,1].mean() / scale_outcome
    swe_beta = trace['beta'][:,0].mean() / scale_outcome
    swe_alpha = trace['alpha'][:,0].mean() / scale_outcome
    nor_beta = trace['beta'][:,2].mean() / scale_outcome
    nor_alpha = trace['alpha'][:,2].mean() / scale_outcome
    den_beta = trace['beta'][:,3].mean() / scale_outcome
    den_alpha = trace['alpha'][:,3].mean() / scale_outcome
    isl_beta = trace['beta'][:,4].mean() / scale_outcome
    isl_alpha = trace['alpha'][:,4].mean() / scale_outcome
    
    
    fin_hdi_kwargs = {'color':'lightblue','alpha':1.0,'label' : 'FIN'}
    swe_hdi_kwargs = {'color' : 'yellow','alpha' : 1.0,'label' : 'SWE'}
    den_hdi_kwargs = {'color' : 'orange','alpha' : 1.0,'label' : 'DNK'}
    nor_hdi_kwargs = {'color' : 'lime','alpha' : 1.0,'label' : 'NOR'}
    isl_hdi_kwargs = {'color' : 'cyan','alpha' : 1.0,'label' : 'ISL'}

    numeric_timeline = df_timeline.reset_index().copy()

    hdi_swe_x = numeric_timeline.loc[numeric_timeline['Country'] == 'SWE']['week_idx']
    hdi_fin_x = numeric_timeline.loc[numeric_timeline['Country'] == 'FIN']['week_idx']
    hdi_den_x = numeric_timeline.loc[numeric_timeline['Country'] == 'DNK']['week_idx']
    hdi_nor_x = numeric_timeline.loc[numeric_timeline['Country'] == 'NOR']['week_idx']
    hdi_isl_x = numeric_timeline.loc[numeric_timeline['Country'] == 'ISL']['week_idx']


    swe_num = numeric_timeline.loc[numeric_timeline['Country'] == 'SWE']
    swe_idx = swe_num.index.values
    hdi_swe_mu = trace['mu'][:,swe_idx]

    fin_num = numeric_timeline.loc[numeric_timeline['Country'] == 'FIN']
    fin_idx = fin_num.index.values
    hdi_fin_mu = trace['mu'][:,fin_idx]

    den_num = numeric_timeline.loc[numeric_timeline['Country'] == 'DNK']
    den_idx = den_num.index.values
    hdi_den_mu = trace['mu'][:,den_idx]

    nor_num = numeric_timeline.loc[numeric_timeline['Country'] == 'NOR']
    nor_idx = nor_num.index.values
    hdi_nor_mu = trace['mu'][:,nor_idx]

    isl_num = numeric_timeline.loc[numeric_timeline['Country'] == 'ISL']
    isl_idx = isl_num.index.values
    hdi_isl_mu = trace['mu'][:,isl_idx]

    fig,axes = plt.subplots(2,3,sharex=True,sharey=True,figsize=(18,12))

    plt.suptitle('Nordic countries : mortality trends for [{}]'.format(outcome) +\
                 ' based on weekly death rates Jan 2007 - Aug 2020')


    axes[0,0].plot(hdi_swe_x,
            (trace['alpha'][0,0].mean() + \
               standardize(numeric_timeline.loc[numeric_timeline['Country'] \
                                           == 'SWE']['week_idx']) * trace['beta'][:,0].mean()) / scale_outcome,
            '--',color='k',lw=1)



    axes[0,0].scatter(df_timeline.loc[df_timeline['Country'] == 'SWE']['week_idx'],
                df_timeline.loc[df_timeline['Country']=='SWE'][outcome],color='lightgrey',alpha=0.3)

    az.plot_hdi(hdi_swe_x,hdi_swe_mu / scale_outcome,hdi_prob=0.89,ax=axes[0,0],fill_kwargs=swe_hdi_kwargs)
    
    axes[0,0].axvline(x.mean(),ls='dashed',color='orange')

    ##

    axes[0,1].plot(hdi_fin_x,
            (trace['alpha'][:,1].mean() + \
               standardize(numeric_timeline.loc[numeric_timeline['Country'] \
                                           == 'FIN']['week_idx']) * trace['beta'][:,1].mean()) / scale_outcome,
            '--',color='k',lw=1)


    axes[0,1].scatter(df_timeline.loc[df_timeline['Country'] == 'FIN']['week_idx'],
                df_timeline.loc[df_timeline['Country']=='FIN'][outcome],color='lightgrey',alpha=0.3)

    az.plot_hdi(hdi_fin_x,hdi_fin_mu / scale_outcome,hdi_prob=0.89,ax=axes[0,1],fill_kwargs=fin_hdi_kwargs)
    
    axes[0,1].axvline(x.mean(),ls='dashed',color='orange')


    ##

    axes[0,2].plot(hdi_den_x,
            (trace['alpha'][:,3].mean() + \
               standardize(numeric_timeline.loc[numeric_timeline['Country'] \
                                           == 'DNK']['week_idx']) * trace['beta'][:,3].mean()) / scale_outcome,
            '--',color='k',lw=1)


    axes[0,2].scatter(df_timeline.loc[df_timeline['Country'] == 'DNK']['week_idx'],
                df_timeline.loc[df_timeline['Country']=='DNK'][outcome],color='lightgrey',alpha=0.3)

    az.plot_hdi(hdi_den_x,hdi_den_mu / scale_outcome,hdi_prob=0.89,ax=axes[0,2],fill_kwargs=den_hdi_kwargs)
    
    axes[0,2].axvline(x.mean(),ls='dashed',color='orange')


    ##
    axes[1,0].plot(hdi_nor_x,
            (trace['alpha'][:,2].mean() + \
               standardize(numeric_timeline.loc[numeric_timeline['Country'] \
                                           == 'NOR']['week_idx']) * trace['beta'][:,2].mean()) / scale_outcome,
            '--',color='k',lw=1)


    axes[1,0].scatter(df_timeline.loc[df_timeline['Country'] == 'NOR']['week_idx'],
                df_timeline.loc[df_timeline['Country']=='NOR'][outcome],color='lightgrey',alpha=0.3)

    az.plot_hdi(hdi_nor_x,hdi_nor_mu / scale_outcome,hdi_prob=0.89,ax=axes[1,0],fill_kwargs=nor_hdi_kwargs)
    
    axes[1,0].axvline(x.mean(),ls='dashed',color='orange')


    ##

    axes[1,1].plot(hdi_isl_x,
            (trace['alpha'][:,4].mean() + \
               standardize(numeric_timeline.loc[numeric_timeline['Country'] \
                                           == 'ISL']['week_idx']) * trace['beta'][:,4].mean()) / scale_outcome,
            '--',color='k',lw=1)


    axes[1,1].scatter(df_timeline.loc[df_timeline['Country'] == 'ISL']['week_idx'],
                df_timeline.loc[df_timeline['Country']=='ISL'][outcome],color='lightgrey',alpha=0.3)

    az.plot_hdi(hdi_isl_x,hdi_isl_mu / scale_outcome,hdi_prob=0.89,ax=axes[1,1],fill_kwargs=isl_hdi_kwargs)
    
    axes[1,1].axvline(x.mean(),ls='dashed',color='orange')


    plt.legend(loc='upper center')

    axes[1,0].set_xlabel('week number')
    axes[1,1].set_xlabel('week_number')
    axes[1,2].set_xlabel('week_number')

    axes[0,0].set_title(r'SWE $\alpha$ : {:.4f}   $\beta$ : {:.6f}'.format(swe_alpha,swe_beta))
    axes[0,1].set_title(r'FIN $\alpha$ : {:.4f}   $\beta$ : {:.6f}'.format(fin_alpha,fin_beta))
    axes[0,2].set_title(r'DNK $\alpha$ : {:.4f}   $\beta$ : {:.6f}'.format(den_alpha,den_beta))
    axes[1,0].set_title(r'NOR $\alpha$ : {:.4f}   $\beta$ : {:.6f}'.format(nor_alpha,nor_beta))
    axes[1,1].set_title(r'ISL $\alpha$ : {:.4f}   $\beta$ : {:.6f}'.format(isl_alpha,isl_beta))

    axes[1,2].axis('off')

    plt.savefig('nordic_mortality_bayesian_trends_{}.jpg'.format(outcome),format='jpg')



In [ ]:
# reported alpha's dont cross at 0 b/c x vals are standardized in calculation : mean at about week 350
plot_reg(outcome)

In [ ]:
df_2020

In [ ]:
age_grps_2020 = df_2020[['Country','age_grp_0-14','age_grp_15-64','age_grp_65-74',
                        'age_grp_75-84','age_grp_85+','all_age_grp']].copy()

age_grps_2020

In [ ]:
country_age_grps = age_grps_2020.groupby('Country')

swe_2020 = country_age_grps.get_group('SWE')
fin_2020 = country_age_grps.get_group('FIN')
nor_2020 = country_age_grps.get_group('NOR')
den_2020 = country_age_grps.get_group('DNK')
isl_2020 = country_age_grps.get_group('ISL')

swe_means = swe_2020.mean()
swe_2020_ci = swe_2020.quantile([0.055,0.945])
swe_2020_ci = np.abs(swe_2020_ci - swe_2020_means)

fin_means = fin_2020.mean()
fin_2020_ci = fin_2020.quantile([0.055,0.945])
fin_2020_ci = np.abs(fin_2020_ci - fin_means)

nor_means = nor_2020.mean()
nor_2020_ci = nor_2020.quantile([0.055,0.945])
nor_2020_ci = np.abs(nor_2020_ci - nor_means)

den_means = den_2020.mean()
den_2020_ci = den_2020.quantile([0.055,0.945])
den_2020_ci = np.abs(den_2020_ci - den_means)

isl_means = isl_2020.mean()
isl_2020_ci = isl_2020.quantile([0.055,0.945])
isl_2020_ci = np.abs(isl_2020_ci - isl_means)

print (swe_means)
print (swe_2020_ci.T)

swe_2020.describe(percentiles=[0.055,0.945])


In [ ]:
all_means = pd.concat([swe_means,fin_means,nor_means,den_means,isl_means],axis=1)
all_means.columns=['SWE','FIN','NOR','DNK','ISL']
all_means

In [ ]:
all_cis = np.array([swe_2020_ci.values,fin_2020_ci.values,nor_2020_ci.values,
                    den_2020_ci.values,isl_2020_ci.values])
all_cis.shape

In [ ]:
all_means

In [ ]:
all_means.mean(axis=1)

In [ ]:
all_means.sub(all_means.mean(axis=1),axis=0)

In [ ]:
all_means.plot(kind='bar',
               figsize=(18,12),yerr=all_cis,error_kw={'capsize' : 5,'ecolor':'k','label' : '89%'},
               title='Nordic Countries 2020 age grp mortality from weekly death rates Jan - Aug\nDataSource : mortality.org')

plt.ylabel('mortality')
plt.yscale('log')
plt.savefig('nordic_mortality_bar_chart.jpg',format='jpg')

In [ ]:
all_means.loc['age_grp_85+'].plot(kind='bar')

In [ ]:
all_means.loc['age_grp_0-14'].plot(kind='bar')